<h1>PRIMERA APROXIMACIÓN AL PROBLEMA<h1>

En la busqueda de un sistema de cifrado homomorfico está motivada en la busqueda de un algoritmo de descifrazo que defina, para una clave privada, un homomorfismo de grupos entre el espacio del mensaje cifrado ($\mathcal{C}$) y el del mensaje sin cifrar ($\mathcal{M}$): 

$$ f: \mathcal{C} \rightarrow \mathcal{M} $$

asi podremos enviar la información cifrada al servicio Cloud, tratarla estando cifrada y posteriormente traer al local las conclusiones para ser descifradas. Para una primera aproximación tomamos el sistema RSA simplificado. 

La clave publica es un par $(N,e)$ donde $N = p \cdot q$ es producto de dos primos $p$ y $q$ suficientemente grandes y distintos. Por otro lado, $e$ es el llamado exponente de cifrado que es un entero positivo tal que mcd $(e,\phi(N)) = 1$, donde $\phi(N) = (p-1)(q-1)$ es la función $\phi$ de Euler. 

La clave privada, es el par $(N,d)$ donde $d$ es el exponente de descifrado (i.e. el inverso de $e$ módulo $\phi(n)$). De este modo se verifica que 

$$ e \cdot d \equiv 1\, (mod\, (p-1)(q-1))$$

así 

$$ \mathcal{M} = \mathcal{C} = (\mathbb{Z}_N)^* $$

que es el grupo multiplicativo de las unidades módulo n, cuyos elementos son 

$$ m \in \mathbb{Z}: 1 \leq m < N \wedge mcd\, (m,n) = 1 $$

quedando el homomorfismo de grupos 

$$ E: (\mathbb{Z}_N)^* \longrightarrow (\mathbb{Z}_N)^* $$
$$ m \mapsto m^e\, mod \, N $$

que es un isomorfismo, que permite descifrar un mensaje cifrado $c$ através de su inverso:  

$$D(c) = c^d \, mod \, N$$

La utilidad de esto radica en que $((\mathbb{Z}_N)^*,\; \cdot)$ es un grupo multiplicativo, por tanto 

$$E(m_1 \cdot m_2) = E(m_1) \cdot E(m_2)$$  

<h2>APLICACIÓN AL DATASET DEL CANCER DE PULMÓN<h2>

En primer lugar vamos a definir dos funciones, una para encriptar un tipo int y otra para desencriptarlo. Los parámetros de entrada para la función que encripta serán 
<ul>
<li>publicKey -> Lista con dos elementos que contiene N, e </li>
<li> -> Lista con dos elementos que contiene N, e </li>


In [1]:
def E_RSA(publicKey, mensaje): 
    n = publicKey[0]
    e = publicKey[1]
    return pow(mensaje, e, n)

def D_RSA(privateKey, criptotexto): 
    n = privateKey[0]
    d = privateKey[1]
    return pow(criptotexto, d, n)